In [18]:
import pickle
from knn import ProjectKNN
from xgmodel import ProjectXGBOOST
from mlp import ProjectMLP, MLP
import torch
import pandas as pd

In [4]:
with open('knn.bin', 'rb') as fiile:
    knn = pickle.load(fiile)
knn.test_arx('LocTreino_Equipe_3.csv')

In [6]:
with open('xgboost.bin', 'rb') as fiile:
    xgboost = pickle.load(fiile)
xgboost.test_arx('LocTreino_Equipe_3.csv')

In [19]:
with open('mlp.bin', 'rb') as fiile:
    mlp = pickle.load(fiile)

test = pd.read_csv('LocTreino_Equipe_3.csv')
x_batch = test[[
    x for x in test.columns if x not in ('lat', 'lon', 'pontoId')]].values
y_batch_x = test[[
    x for x in test.columns if x in ('lat')]].values
y_batch_y = test[[
    x for x in test.columns if x in ('lon')]].values

x_batch = torch.tensor(x_batch).float().cuda()
y_batch_x = torch.tensor(y_batch_x).float().cuda()
y_batch_y = torch.tensor(y_batch_y).float().cuda()

mlp.model_x.eval()
out_x = mlp.model_x(x_batch)
out_x = out_x.detach().cpu().numpy()

mlp.model_y.eval()
out_y = mlp.model_y(x_batch)
out_y = out_y.detach().cpu().numpy()

df = pd.DataFrame()
df['pontoId'] = test.pontoId
df['lat'] = y_batch_x.cpu()
df['lon'] = y_batch_y.cpu()
df['lat_pred'] = out_x
df['lon_pred'] = out_y
df.to_csv('Resultados_Equipe3_Metodo_MLP.csv', index=False)

TypeError: can't convert CUDA tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.